## Import Dependencies

In [5]:
import numpy as np
import numpy.linalg as la

## Generate Data

In [7]:
d = 100 # Number of features
m = 50 # Output dimension
n = 1000  # Number of samples
rank = 10  # Rank of the matrix W
n_agents = 3 # Number of agents

# Generating n input data points with d features with a normal distribution
X = np.random.randn(n, d)

# Normalizing the rows of X
X = X / la.norm(X, axis=1, keepdims=True)

# Generating a random matrix W with rank r
e = 1
S_star = [1]

# Generating the eigenvalues of W
eigen_gaps = [10 for i in range(rank-1)]

for i in range(rank-1):
    e = e + eigen_gaps[i]
    S_star.insert(0,e)

M = np.random.randn(d, m)
[U, _, V] = la.svd(M, full_matrices=False)

A_star = U[:, :rank] @ np.diag(np.sqrt(S_star))
B_star = np.diag(np.sqrt(S_star)) @ V[:rank, :]

W_star = A_star @ B_star

# Generating the output data Y
Y_star = X @ W_star

# Adding noise to the output data
Y = Y_star + 0.1 * np.random.randn(n, m)

# Splitting the data into n_agents parts
X_split = np.array_split(X, n_agents)
Y_split = np.array_split(Y, n_agents)

# Saving the data
np.save('data/X.npy', X)
np.save('data/Y.npy', Y)
np.save('data/W_star.npy', W_star)
np.save('data/A_star.npy', A_star)
np.save('data/B_star.npy', B_star)

print(X.shape)
print(Y.shape)
print(W_star.shape)


(1000, 100)
(1000, 50)
(100, 50)


# Measuring similarities and subspace overlap

In [8]:
class RegressionAgent:
    def __init__(self, d, m):
        self.d = d
        self.m = m
        self.W = None
    
    def init_weights(self, W=None):
        if W is None:
            self.W = np.random.randn(self.d, self.m)
        else:
            self.W = W
        
    def forward(self, X):
        return X @ self.W
    
    def loss(self, X, Y):
        return 0.5 * la.norm(Y - self.forward(X))**2
    
    def update_weights(self, X, Y, lr=0.01):
        self.W = self.W - lr * (X.T @ (X @ self.W - Y))
    
# True model for reference
true_agent = RegressionAgent(d, m)
true_agent.init_weights(W_star)

print("True loss: ", true_agent.loss(X, Y))

True loss:  251.5761037285171


In [15]:
agents = [RegressionAgent(d, m) for i in range(n_agents)]

# W_init = np.random.randn(d, m)

for i in range(n_agents):
    # agents[i].init_weights(W_init)
    agents[i].init_weights()

print("Agent 0: Initial loss on the whole dataset:", agents[0].loss(X, Y))
print("Agent 1: Initial loss on the whole dataset:", agents[1].loss(X, Y))
print("Agent 2: Initial loss on the whole dataset:", agents[2].loss(X, Y))

Agent 0: Initial loss on the whole dataset: 172312.10713647128
Agent 1: Initial loss on the whole dataset: 170330.52680617437
Agent 2: Initial loss on the whole dataset: 170806.12222502741


In [16]:
def train_W(agent, X, Y, lr=0.001, epochs=100):
    loss = []
    print("Training agent")
    for i in range(1, epochs + 1):
        agent.update_weights(X, Y, lr)
        loss.append(agent.loss(X, Y))
        if i % 100 == 0:
            print(f"Epoch {i}: {loss[-1]}")
    return loss


losses = [
    train_W(agents[i], X_split[i], Y_split[i], 1e-2, 1000) 
    for i in range(n_agents)
]

Training agent
Epoch 100: 565.1210336897396
Epoch 200: 103.66253397299697
Epoch 300: 65.88431048197418
Epoch 400: 61.10528192652962
Epoch 500: 60.38467031863687
Epoch 600: 60.264809127330274
Epoch 700: 60.24347986708611
Epoch 800: 60.239484464540226
Epoch 900: 60.23870531493153
Epoch 1000: 60.23854849519536
Training agent
Epoch 100: 582.9260835482628
Epoch 200: 102.26463447809303
Epoch 300: 64.08921123939476
Epoch 400: 59.301600369290846
Epoch 500: 58.56253472064687
Epoch 600: 58.43404066784907
Epoch 700: 58.410016640395874
Epoch 800: 58.405315410930704
Epoch 900: 58.40436791501842
Epoch 1000: 58.404173141462735
Training agent
Epoch 100: 598.4137589651317
Epoch 200: 106.04108900338254
Epoch 300: 64.58404724249752
Epoch 400: 59.43959196534426
Epoch 500: 58.691465103037935
Epoch 600: 58.572735946489225
Epoch 700: 58.55277942609558
Epoch 800: 58.5492813834573
Epoch 900: 58.54864791908056
Epoch 1000: 58.54853014671986


In [17]:
print("Agent 0: Loss on the whole dataset:", agents[0].loss(X, Y))
print("Agent 1: Loss on the whole dataset:", agents[1].loss(X, Y))
print("Agent 2: Loss on the whole dataset:", agents[2].loss(X, Y))

Agent 0: Loss on the whole dataset: 297.67939923535107
Agent 1: Loss on the whole dataset: 297.7461632624849
Agent 2: Loss on the whole dataset: 299.78394180214644


In [20]:
def federated_average_W(agents):
    W = np.zeros((d, m))
    for agent in agents:
        W += agent.W
    W /= len(agents)
    return W

W_avg = federated_average_W(agents)
model_agg = RegressionAgent(d, m)
model_agg.init_weights(W_avg)

print("Model aggregation loss: ", model_agg.loss(X, Y))

Model aggregation loss:  234.20915733911215


---

In [29]:
def subspace_overlap(W1, W2):
    U1, _, _ = la.svd(W1)
    U2, _, _ = la.svd(W2)
    return la.norm(U1.T @ U2, ord='fro')**2

(100, 50)
(100, 50)
(100, 100)
(100, 100)
Distance between true model and model aggregation:  9.999999999999998


: 